In [ ]:
import os
import cv2
import random
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
import torchvision.models as models

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Paths to dataset
DATASET_PATH = r"D:\New_Project\DataSet"  # Replace with your dataset path

# 1. Dataset Handling
class VideoDataset(Dataset):
    def __init__(self, video_paths, labels, transform=None, max_frames=100):
        self.video_paths = video_paths
        self.labels = labels
        self.transform = transform
        self.max_frames = max_frames

    def __len__(self):
        return len(self.video_paths)

    def __getitem__(self, idx):
        cap = cv2.VideoCapture(self.video_paths[idx])
        frames = []
        count = 0
        while cap.isOpened() and count < self.max_frames:
            ret, frame = cap.read()
            if not ret:
                break
            frame = cv2.resize(frame, (224, 224))
            if self.transform:
                frame = self.transform(frame)
            frames.append(frame)
            count += 1
        cap.release()
        return torch.stack(frames), self.labels[idx]

def load_dataset(base_path):
    video_paths, labels = [], []
    
    # Handling real videos from two folders
    real_folders = ["Celeb-real", "YouTube-real"]
    fake_folder = "Celeb-synthesis"
    
    for folder in real_folders:
        folder_path = os.path.join(base_path, folder)
        for file in os.listdir(folder_path):
            if file.endswith(".mp4") or file.endswith(".avi"):
                video_paths.append(os.path.join(folder_path, file))
                labels.append(0)  # Real videos labeled as 0

    # Handling fake videos
    folder_path = os.path.join(base_path, fake_folder)
    for file in os.listdir(folder_path):
        if file.endswith(".mp4") or file.endswith(".avi"):
            video_paths.append(os.path.join(folder_path, file))
            labels.append(1)  # Fake videos labeled as 1
    
    return video_paths, labels

# 2. Model Definition
class DeepFakeDetectionModel(nn.Module):
    def __init__(self):
        super(DeepFakeDetectionModel, self).__init__()
        self.resnext = models.resnext50_32x4d(pretrained=True)
        self.resnext.fc = nn.Identity()  # Remove fully connected layer
        self.lstm = nn.LSTM(input_size=2048, hidden_size=512, num_layers=2, batch_first=True)
        self.fc = nn.Linear(512, 1)  # Binary classification

    def forward(self, x):
        batch_size, num_frames, _, _, _ = x.size()
        x = x.view(batch_size * num_frames, 3, 224, 224)
        x = self.resnext(x)
        x = x.view(batch_size, num_frames, -1)
        lstm_out, _ = self.lstm(x)
        lstm_out = lstm_out[:, -1, :]
        return torch.sigmoid(self.fc(lstm_out))

# 3. Training and Evaluation
def train(model, train_loader, optimizer):
    model.train()
    total_loss = 0
    for frames, labels in train_loader:
        frames, labels = frames.to(device), labels.to(device)
        outputs = model(frames)
        loss = nn.BCELoss()(outputs, labels.unsqueeze(1).float())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(train_loader)

def evaluate(model, loader):
    model.eval()
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for frames, labels in loader:
            frames, labels = frames.to(device), labels.to(device)
            outputs = model(frames)
            preds = (outputs > 0.5).float()
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    return accuracy_score(all_labels, all_preds), confusion_matrix(all_labels, all_preds)

# 4. Main Function for Video Prediction
def predict_video(model, video_path):
    transform = transforms.Compose([transforms.ToTensor()])
    cap = cv2.VideoCapture(video_path)
    frames = []
    count = 0
    while cap.isOpened() and count < 100:
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.resize(frame, (224, 224))
        frame = transform(frame)
        frames.append(frame)
        count += 1
    cap.release()
    if len(frames) == 0:
        return "Error: No frames detected."
    with torch.no_grad():
        model.eval()
        frames = torch.stack(frames).unsqueeze(0).to(device)
        output = model(frames)
        confidence = output.item()
        label = "Fake" if confidence > 0.5 else "Real"
        return f"Prediction: {label}, Confidence: {confidence:.2f}"

# 5. Run Training and Prediction
if __name__ == "__main__":
    video_paths, labels = load_dataset(DATASET_PATH)
    
    train_videos, test_videos, train_labels, test_labels = train_test_split(video_paths, labels, test_size=0.2, random_state=42)
    val_videos, test_videos, val_labels, test_labels = train_test_split(test_videos, test_labels, test_size=0.5, random_state=42)

    train_dataset = VideoDataset(train_videos, train_labels, transform=transforms.ToTensor())
    val_dataset = VideoDataset(val_videos, val_labels, transform=transforms.ToTensor())
    test_dataset = VideoDataset(test_videos, test_labels, transform=transforms.ToTensor())

    train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=4, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=4, shuffle=False)

    model = DeepFakeDetectionModel().to(device)
    optimizer = optim.Adam(model.parameters(), lr=1e-4)

    for epoch in range(5):  # Adjust epochs as needed
        train_loss = train(model, train_loader, optimizer)
        val_accuracy, val_conf_matrix = evaluate(model, val_loader)
        print(f'Epoch {epoch+1}: Loss = {train_loss:.4f}, Val Accuracy = {val_accuracy:.4f}')
        print('Confusion Matrix:', val_conf_matrix)

    # Test a video upload
    test_video_path = r"D:\New_Project\Uploaded_Videos\download.mp4"  # Replace with your test video path
    print(predict_video(model, test_video_path))


C:\Users\gargd\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\gargd\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNeXt50_32X4D_Weights.IMAGENET1K_V1`. You can also use `weights=ResNeXt50_32X4D_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
